In [3]:
import os
import sys
import pickle
from packages import data_container
from packages.data_container import Data
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [14]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 
      'w_goal': 0.4, 'w_obst': 0.2, 'ps':[], 'time_stamp': []}
exp = Data(Hz=90, header=header, info=info)
for subj_id in range(1):
    # Import output
    file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw Data', 'Subj' + str(subj_id).zfill(2) + '.csv'))
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
        head = 0
        while head + 1 < len(trials):
            for i in range(head+1, len(trials)):
                if trials[i][0] == '':
                    tail = i - 1
                    break
            trial_id = int(trials[head+1][1])
            angle = float(trials[head+1][3])
            speed = float(trials[head+1][5])
            subj = [[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]]
            goal = [[float(line[-2]), float(line[-1])] for line in trials[head+3:tail+1]]
            obst = [[float(line[-4]), float(line[-3])] for line in trials[head+3:tail+1]]
            time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
            for i in range(len(obst) - 1):
                if obst[i][0] != obst[i + 1][0] or obst[i][1] != obst[i + 1][1]:
                    onset = i
                    break
            exp.add_traj(subj)
            exp.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_speed':speed, 'subj_id':subj_id, 
                       'trial_id':trial_id, 'stimuli_onset':onset, 'time_stamp':time_stamp})
            head = tail + 1
            
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw Data', 'Bai_movObst1_data.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(exp, f, pickle.HIGHEST_PROTOCOL)  

In [13]:
exp.info['stimuli_onset'][0]

1059